# Chapter 5: 一等函数

Python 并不是functional programming language --- python 之父Guido Van Rossum

Function 在python 中是first-class objects.

**一等对象**的定义:
- 在运行时创建
- 能复制给变量或数据结构中的元素
- 能作为参数传给函数
- 能作为函数返回结果

在Python 中，函数是一种一等对象，类似于整数，字符串，字典等。

"treating functions as objects".

## 1. 把函数视为对象

下面，我们首先定义一个函数，然后通过type 命令看出，这个函数是一个function 类的示例。

In [4]:
def factorial(n):
    '''returns n'''
    return 1 if n < 2 else n * factorial(n - 1)

In [5]:
factorial(42)

1405006117752879898543142606244511569936384000000000

下面我们来看，一个function 其实是一个对象。
- 调用其`__doc__`方法: used to generate the **help text** of an object.
- 查看type

In [6]:
print(factorial.__doc__)

returns n


In [7]:
print(type(factorial))

<class 'function'>


下面，我们来展示函数对象的一等特性
#### 赋值给变量

In [8]:
fact = factorial
print(fact)

<function factorial at 0x10fabe400>


In [9]:
fact(42)

1405006117752879898543142606244511569936384000000000

#### 作为参数传参

In [10]:
lst = [i for i in range(10)]
lst

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
list(map(factorial, lst))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

`map` 方法返回一个iterable：每个元素都是第一个参数（方法）应用在第二个参数的每个元素上的结果。

上述操作可以描述为：对每一个lst 中的元素，使用factorial 方法。

要现实一个iterable，要将其转化为list.

## 2. 高阶函数 (Higher Order Functions)

有了一等函数，就可以使用函数式风格编程。函数式编程的特点之一就是使用高阶函数。

#### 高阶函数（Higher-Order Function）: 接受函数为参数，或者把函数作为结果返回的函数。

上面的map 函数就是一个例子。还有一个普遍使用的例子是sorted, 接受一个函数作为key，表明排序的依据。例如，下面我们根据长度对字符串进行排序。

In [12]:
fruits = ['apple', 'orange', 'respberry']
sorted(fruits, key=len)

['apple', 'orange', 'respberry']

同样，我们可以根据reversed spelling 排序

In [13]:
def reverse(word):
    return word[::-1]

In [14]:
reverse('abc')

'cba'

In [16]:
sorted(fruits, key=reverse)  # sorted by reversed spelling

['orange', 'apple', 'respberry']

最常用的高阶函数有：
- map
- filter
- reduce
- apply (Python 3 已经移除)
    - function with dynamic set of arguments
    - before: `apply(fn, args, kwargs)`
    - now: `fn(*args, **kwargs)`

`map` 和 `filter` 的功能可以由列表推导式和生成器表达式替代。使用列表推倒式的好处是：可读性更好。

例如，下面我们来看列表推导式和map, filter 的区别。

In [17]:
list(map(factorial, lst))  # map 

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

In [18]:
[factorial(n) for n in lst]  # 列表推导式 

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

In [19]:
list(map(factorial, filter(lambda n : n % 2, lst)))  # filter

[1, 6, 120, 5040, 362880]

In [20]:
[factorial(n) for n in lst if n % 2]  # 列表推导式

[1, 6, 120, 5040, 362880]

Python 3 中map 和filter 返回一个生成器，所以需要用list 方法获取全量的元素。

`reduce` 函数在python 2中是内置函数，在python 3中放在了`functools` package 中。

In [19]:
from functools import reduce
from operator import add
reduce(add, lst)

45

In [21]:
sum(lst)  # 上述操作等同于sum 求和

45

#### 规约函数
- all: 如果iterable 的每个元素都是真值，返回True
    - `all([])` 返回True
- any: 如果iterable 中有元素是真值，返回True
    - `any([])` 返回False
    
⚠️ 第14章会介绍可迭代对象以及规约函数，第10章会详细介绍reduce 函数。

## 3. 匿名函数

有时创建小的一次性(small and one-off function) 的函数十分遍历，这就是匿名函数，`lambda`

注意，lambda 的定义中不能赋值，不能使用while，try，等python 语句。

**匿名函数的最佳使用场景**: lambda 常用与函数的参数中. 

Lambda 函数的limitation：
- 因为只能使用pure expression，因此表述力不高
- 可读性不好

因此lambda 函数在其他场景下很少使用。

In [21]:
sorted(fruits, key=lambda word: word[::-1])  # 反转之后排序

['orange', 'apple', 'respberry']

## 4. 可调用对象

使用lambda 和def 都会创建一个函数对象。函数对象是一种可调用对象。Python 一共有7种可调用（callable）对象：
- 用户定义函数：`def` / `lambda`
- 内置函数：使用C语言实现的函数，例如`len`
- 内置方法：使用C语言实现的方法，例如`dict.get`
- 方法：在类中定义的函数
- 类：先调用`__new__`方法创建实例，再调用`__init__`方法初始化实例
- 类的实例：如果类定义了`__call__`方法，那么它的实例可以作为函数调用 --- 参见Part 5
- 生成器函数：yield 关键字定义的函数或方法，返回生成器对象(generator object)。
    - 生成器函数还可以作为协程，参见16章

#### what is the difference between method and function?
方法是指在类中定义的函数。

## 5. 用户定义的可调用类型

通过实现`__call__` 方法，可以让对象表现的像函数。**function-like object**

In [23]:
class Student:
    def __init__(self, first_name, last_name, age, department):
        self._first_name = first_name
        self._last_name = last_name
        self._age = age
        self._department = department
        
    def get_name(self):
        return self._first_name + ' ' + self._last_name
    
    def __call__(self):
        return self.get_name()  

In [25]:
stu = Student('Chen', 'Wang', 35, 'CS')
print(stu.get_name())
print(stu())

Chen Wang
Chen Wang


In [26]:
callable(stu)  # 通过内置的callable 方法判定它是否为可调用对象

True

上面的例子，名字可能会被修改，这样我们在每次调用`stu()`的时候可能返回不同的结果。相当于是一个有状态的function。类似的还有修饰器(decorator) 和闭包(clusure). 会在第7章介绍。

## 6. 函数内省（Function Introspection）

下面我们介绍另一个把函数作为对象的方面，runtime introspection.

前面我们已经介绍过函数作为对象的`__doc__` 属性，下面，我们使用`dir()`方法来看看函数作为对象的所有属性。

In [27]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

大多数是Python 对象共有的属性，下面我们着重讲一些与函数对象特有的属性。我们首先获取函数的属性集合，然后获取一般对象的属性集合，最后做一个差值。

In [29]:
sorted(set(dir(factorial)) - set(dir(Student)))

['__annotations__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

我们汇总一下，一共有以下几种方法，我们后面会陆续介绍。

| 名称 | 类型 | 说明 |
|------|------|------|
|__annotations__| dict | Parameter and return annotations|
|__call__| method-wrapper | Implementation of the () operator; a.k.a. the callable object protocol|
|__closure__| tuple | The function closure, i.e., bindings for free variables (often is None)|
|__code__| code | Function metadata and function body compiled into bytecode|
|__defaults__| tuple | Default values for the formal parameters|
|__get__| method-wrapper | Implementation of the read-only descriptor protocol (see Chapter 20)|
|__globals__| dict | Global variables of the module where the function is defined|
|__kwdefaults__| dict | Default values for the keyword-only formal parameters|
|__name__| str | The function name|
|__qualname__| str | The qualified function name, e.g., Random.choice (see PEP-3155)|

## 7. 从定位参数到仅限关键字参数（From Positional to Keyword-Only Parameters）

Python 的一个强大的地方是：flexible parameter handling mechanism, enhanced with keyword-only arguments.

Python 的函数有两类参数：
- **positional parameters**: must be included in the correct order. 
- **keyword only parameters**: included with a keyword and equals sign

假设我们有一个函数签名

```python
def my_func(name, *content, cls=None, **attrs)
```

- `my_func('a name')` - single positional argument 匹配第一个参数name
- `my_func('a name', 'hello', 'world')` - 从第二个positional argument 被`*content` 捕获, 作为tuple
- `my_func('a name', 'hello', id=01)` - Keyword arguments not explicitly named in function signature are captured by `**attrs` as a dict.
- `my_func('a name', 'hello', 'world', cls='sidebar')` - The cls parameter can only be passed as a keyword argument.
- `my_func(content='testing', name="img")` - 第一个positional argument 也可以作为keyword 
- `my_func(*my_dict)` - 如果有named parameter 则被捕获，否则被attr 捕获

可以理解成：
- `*content`: 捕获多个positional parameters
- `**attrs`: 捕获多个unnamed keyword parameters

#### 使用一个* 

In [28]:
def f(a, *, b):
    return a, b

In [30]:
f(1, 2)

TypeError: f() takes 1 positional argument but 2 were given

In [31]:
f(1, b=2)

(1, 2)

In [32]:
f(1,2,b=3)

TypeError: f() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given

## 8. Retrieving Information About Parameters

Bobo is a light-weight framework for creating WSGI web applications.

In [50]:
def my_func(a, b=2, *c, d=3, **e):
    local_var = 0
    return a, b, c, d, e, local_var

#### `__defaults__` positional and keyword arguments

In [51]:
my_func.__defaults__

(2,)

#### `__kwdefaults__` keyword-only arguments


In [52]:
my_func.__kwdefaults__

{'d': 3}

In [53]:
my_func.__code__

<code object my_func at 0x10fabf8a0, file "<ipython-input-50-82b0069dfb89>", line 1>

In [54]:
my_func.__code__.co_varnames

('a', 'b', 'd', 'c', 'e', 'local_var')

可以看出，同时保留了函数创建的本地变量`local_var`.

In [55]:
my_func.__code__.co_argcount

2

## 9. 函数注解(Function Annotations)


In [5]:
def add_two_num(a, b=10):
    return a+b

In [6]:
add_two_num(1,2)

3

In [7]:
add_two_num(1)

11

下面我们为上述函数添加注释。
- 每个参数可以在":"后添加注解表达式
- 如果有默认值，直接表达式放在参数名和=之间
- 直接返回值，在) 和: 直接加一个->

注解表达式通常为：
- 类（例如str，int 等）
- 字符串（例如int >0 等）

In [9]:
def another_add_two_num(a:int, b:int=10) -> int:
    return a+b

In [10]:
another_add_two_num(1,2)

3

In [11]:
another_add_two_num(1)

11

注解只是愿数据，编译器什么都不会做。annotation 不会做特殊处理，只会在函数的`__annotation__` 属性中。

In [12]:
another_add_two_num.__annotations__

{'a': int, 'b': int, 'return': int}

In [13]:
add_two_num.__annotations__

{}

函数注解最大的用处在于，为IDE 等工具提供静态类型检查等额外信息。

## 10. Packages for Functional Programming (支持函数式编程的包)

### 10.1 Operator 模块

In [14]:
from functools import reduce

def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

In [15]:
fact(3)

6

In [16]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

In [17]:
fact(3)

6

## 11. Chapter Summary